In [1]:
#adding module
import pandas as pd
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pydotplus
import pydot
from io import StringIO
import re

In [2]:
#get info
diabet_info = pd.read_csv("diabetes.csv")
X = diabet_info.values[:, 0:5]
Y = diabet_info.values[:, 5]
n = len(list(diabet_info.columns))
feature_class = list(diabet_info.columns)[0:n-1]

In [3]:
#Splitting sample in to test_set and training_test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, splitter = 'best') #using gini
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(random_state=100)

In [4]:
#test accuracy
from sklearn.metrics import accuracy_score
def prediction(X_test, clf_object): 
    y_pred = clf_gini.predict(X_test)
    return y_pred

def cal_accuracy(y_test, y_pred): 
    print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 
    
y_pred_gini = prediction(X_test, clf_gini) 
cal_accuracy(y_test, y_pred_gini)

Accuracy :  72.72727272727273


In [5]:
#visualizing
dotfile = StringIO()

export_graphviz(clf_gini, out_file=dotfile, feature_names=feature_class, class_names = ['No','Yes'], rounded=True, filled=True)
(graph_gini,) = pydot.graph_from_dot_data(dotfile.getvalue())
graph_gini.write_png("diabet_before_prunning.png")

In [6]:
#prunning the tree
import copy
prune_clf_gini = copy.deepcopy(clf_gini)

def prune_gini(decisiontree, min_gini = 0):
    if decisiontree.min_impurity_split == None:
        decisiontree.min_impurity_split = 0
    decisiontree.min_impurity_split = min_gini
    tree = decisiontree.tree_
    for i in range(tree.node_count):
        if tree.impurity[i] < min_gini:
            tree.children_left[i] = -1
            tree.children_left[i] = -1
            tree.impurity[i] = 0

prune_gini(prune_clf_gini, 0.2)

In [7]:
#visualizing
dotfile2 = StringIO()

export_graphviz(prune_clf_gini, out_file=dotfile2, feature_names=feature_class, class_names = ['No','Yes'], rounded=True, filled=True)
(prune_graph_gini,) = pydot.graph_from_dot_data(dotfile2.getvalue())
prune_graph_gini.write_png("diabet_after_post-prunning.png")

In [8]:
#pre-prunning using sklearn
preprune_clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, splitter = 'best', min_impurity_split = 0.2)
preprune_clf_gini.fit(X_test, y_test)

C:\Users\Meow\Anaconda3\lib\site-packages\sklearn\tree\_classes.py:310: FutureWarning: The min_impurity_split parameter is deprecated. Its default value has changed from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  FutureWarning)


DecisionTreeClassifier(min_impurity_split=0.2, random_state=100)

In [9]:
#visualizing
dotfile3 = StringIO()

export_graphviz(preprune_clf_gini, out_file=dotfile3, feature_names=feature_class, class_names = ['No','Yes'], rounded=True, filled=True)
(preprune_graph_gini,) = pydot.graph_from_dot_data(dotfile3.getvalue())
preprune_graph_gini.write_png("diabet_after_pre-prunning.png")

In [10]:
#test accuracy
y_pred_gini_preprune = prediction(X_test, preprune_clf_gini) 
cal_accuracy(y_test, y_pred_gini_preprune)

Accuracy :  72.72727272727273


In [ ]:
y_pred_gini_postprune = prediction(X_test, preprune_clf_gini) 
cal_accuracy(y_test, y_pred_gini_preprune)